# System Information

Autogenerated info about each system

In [ ]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

import json, pprint

In [ ]:

for path in modules.utils.find_run_outputs(root='../../output', test='Sysinfo', ext='.json'):
    with open(path) as f:
        sysinfo = json.load(f)
        pprint.pprint(sysinfo)
